In [1]:
import math
import pandas as pd
import altair as alt
import numpy as np
from src.spinorama.load_parse import parse_all_speakers, parse_graphs_speaker
from src.spinorama.graph import radar_params_default

df1 = parse_graphs_speaker('Adam', 'Adam S2V', 'klippel')
df2 = parse_graphs_speaker('Klipsch', 'Klipsch R41M', 'klippel')
df3 = parse_graphs_speaker('JBL', 'JBL 104', 'klippel')
df4 = parse_graphs_speaker('Genelec', 'Genelec 8341A', 'klippel')
df5 = parse_graphs_speaker('Sanders\ Sound\ Systems', 'Sanders\ Sound\ Systems\ Model\ 11', 'princeton')

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
def normalize1(df):
    for k in df.keys():
        if k == 'Freq' or k == 'On Axis':
            continue
        df[k] -= df['On Axis']
    df['On Axis'] = 0
    return df

def normalize2(df):
    return df-df.mean()

def directivity(df):    
    r = np.floor(np.logspace(1.0+math.log10(2), 4.0+math.log10(2), 200))
    x, y = np.meshgrid(r, r)
    
    splV = df['SPL Vertical_unmelted'].set_index('Freq')
    splH = df['SPL Horizontal_unmelted'].set_index('Freq')
    
    z = splV.dot(splH.T)/np.sqrt(splV.dot(splV.T) * splH.dot(splH.T))-1

    source = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(),'z': z.melt().value})

    return alt.Chart(source).mark_rect().encode(
        x=alt.X('x:O', scale=alt.Scale(type='log')),
        y=alt.Y('y:O', scale=alt.Scale(type='log')),
        color=alt.Color('z:Q', scale=alt.Scale(scheme='spectral', nice=True))
    ).properties(width=200, height=200)

g1 = directivity(df1).properties(title='Adam S2V')
g2 = directivity(df2).properties(title='Klipsch R41M')
g3 = directivity(df3).properties(title='JBL 104')
g4 = directivity(df4).properties(title='Genelec 8341A')
#g5 = directivity(df5)
(g1 | g2) & (g3 | g4)
#g3

In [ ]:
a = [[1,2], [3,4], [5,6]]
at = np.transpose(a)
ad = np.dot(a, at)
a2 = np.linalg.norm(ad)
ad / a2